# Question 3

We import norm from scipy.stats which will help us in the Normal distribution of the Black & Scholes option valuation. We import njit from numba to fasten the process and see the timing difference.

In [ ]:
from math import log, exp, sqrt
from scipy.stats import norm
from numba import njit
import random
import numpy as np

We develop the DigitalCallOptionAnalytical function which takes the given values S0 (Stock Price), K (Strike Price), T (Time to expiration), r (risk-free rate), q (dividend rate), sigma (volatility) and calculate the Digital call option value.

In [ ]:
def DigitalCallOptionAnalytical(S0,K,T,r,q,sigma):
    """
    Calculates digital call option using analytical formula
    """
    d2 = (log(S0/K) + (r - q - sigma**2/2.0)*T) /sigma*sqrt(T) 
    value = exp(-r*T)* norm.cdf(d2)
    return value

Below, we have the Digital call option valuation using the MonteCarlo simulation. We put the initial payoff as zero. Then, we build a loop using the range function which is used to set the number of simulations to be performed. We set the z variable using the random.gauss function to create a random gaussian variable. Then, we define the Digital payoff using an if-elif statement which returns 1 if the stock price is higher then the strike price and 0 otherwise. The payoff is then added to the initial payoff variable which is 0. Lastly, after the loop the payoff is calculated, we calculate the option value using the digital option formula.

In [ ]:
def DigitalCallOptionMC(S0,K,T,r,q,sigma,numPaths):
    """
    Calculates digital call option using Monte Carlo
    """
    payOff = 0.0
    for i in range(0,numPaths): # Loop that goes for the numPaths set
        z = random.gauss(0.0,1.0)
        S = S0 * exp((r-q-sigma**2/2.0) * T + sigma * sqrt(T) * z)
        def DigitalPayoff(S,K):
            if S-K>=0.0:
                return 1.0
            else:
                return 0.0
        payOff += DigitalPayoff(S,K)
    value = payOff * exp(-r*T) / numPaths
    return value

Below, we define the Digital call option MonteCarlo formula using Numpy. Therefore, compared to the above formula, we assign to the z variable the random gaussian variable using the Numpy function np.random.normal. We change also the exp, sqrt and sum functions with the equivalent Numpy ones. Finally, for the payoff we use the heaviside function, part of Numpy, which passes 1 if the difference between the stock price and the strike price is greater than 1, otherwise it passes 0.

In [ ]:
def DigitalCallOptionMC_Numpy(S0,K,T,r,q,sigma,numPaths):
    """
    Calculates digital call option using NumPy
    """
    z = np.random.normal(size=numPaths,loc=0.0,scale=1.0) 
    S = S0 * np.exp((r-q-sigma**2/2.0) * T + sigma * np.sqrt(T) * z)
    payOff = np.heaviside(S-K,1) # This function shows the payoff of 1 is S-K is greater than zero
    value = np.sum(payOff) * np.exp(-r*T) / numPaths
    return value

Below, we have the same DigitalCallOptionMC formula. However, we add the @njit at the beginning, allowing us to calculate the whole definition using Numba. This should display a faster computation.

In [ ]:
@njit # It performs the below formula using Numba
def DigitalCallOptionMC_Numba(S0,K,T,r,q,sigma,numPaths):
    """
    Calculates digital call option using Numba
    """
    payOff = 0.0
    for i in range(0,numPaths):
        z = random.gauss(0.0,1.0)
        S = S0 * exp((r-q-sigma**2/2.0) * T + sigma * sqrt(T) * z)
        def DigitalPayoff(S,K):
            if S-K>=0.0:
                return 1.0
            else:
                return 0.0
        payOff += DigitalPayoff(S,K)
    value = payOff * exp(-r*T) / numPaths
    return value

Below, the values for each variable we will use to calculate the option value using the different definitions.

In [ ]:
S0, K, T, r, q, sigma, numPaths = 110, 100, 1.0, 0.05, 0.0, 0.4, 10000

In [ ]:
DigitalCallOptionAnalytical(S0,K,T,r,q,sigma)

In [ ]:
DigitalCallOptionMC(S0,K,T,r,q,sigma,numPaths)

In [ ]:
DigitalCallOptionMC_Numpy(S0,K,T,r,q,sigma,numPaths)

In [ ]:
DigitalCallOptionMC_Numba(S0,K,T,r,q,sigma,numPaths)

In [ ]:
%timeit DigitalCallOptionAnalytical(S0,K,T,r,q,sigma)

In [ ]:
%timeit DigitalCallOptionMC(S0,K,T,r,q,sigma,numPaths)

In [ ]:
%timeit DigitalCallOptionMC_Numpy(S0,K,T,r,q,sigma,numPaths)

In [ ]:
%timeit DigitalCallOptionMC_Numba(S0,K,T,r,q,sigma,numPaths)

Using the %timeit function we can see the time to calculate the Digital Call Option using the different definitions. As can we see the Numba function appears to be faster than the Numpy one. However, using Monte Carlo is slower than both of them, whilst the analytical formula is the fastest one.